## Step 1: Extract from Database

Set the database paramaters as environment variables: do NOT enter them here.

In [ ]:
import psycopg2
import os

_POSTGRES_DB_NAME = os.environ["CONTENT_CURATION_POSTGRES_DB_NAME"]
_POSTGRES_DB_USER = os.environ["CONTENT_CURATION_POSTGRES_USER"]
_POSTGRES_DB_PASS = os.environ["CONTENT_CURATION_POSTGRES_PASSWORD"]
_POSTGRES_DB_HOST = os.environ["CONTENT_CURATION_POSTGRES_HOST"]
_POSTGRES_DB_PORT = os.environ["CONTENT_CURATION_POSTGRES_PORT"]

POSTGRES_DB_URL = f'postgres://{_POSTGRES_DB_USER}:{_POSTGRES_DB_PASS}@{_POSTGRES_DB_HOST}:{_POSTGRES_DB_PORT}/{_POSTGRES_DB_NAME}'

In [ ]:
day_seconds = 24*60*60
week_seconds = day_seconds * 7
month_seconds = day_seconds * 30
year_seconds = day_seconds * 365
season_seconds = year_seconds // 4

In [ ]:
recent_topic_seconds = week_seconds
old_topic_seconds = 2*week_seconds

In [ ]:
import time
current_time = time.time()

### Collect Candidate N-Grams

In [ ]:
with psycopg2.connect(POSTGRES_DB_URL) as conn:
    cur = conn.cursor()

    cur.execute("""
        SELECT COUNT(*)
        FROM doc_freq NATURAL JOIN social_post_data
        WHERE create_utc >= %s AND create_utc < %s;
    """, (current_time - recent_topic_seconds, current_time))
    num_recent, = cur.fetchone()
    cur.execute("""
        SELECT COUNT(*)
        FROM doc_freq NATURAL JOIN social_post_data
        WHERE create_utc >= %s AND create_utc < %s;
    """, (current_time - old_topic_seconds, current_time - recent_topic_seconds))
    num_old, = cur.fetchone()
num_old,num_recent

In [ ]:
from dataclasses import dataclass
from typing import List

def get_batch_freq(conn: psycopg2.extensions.connection, ngrams : List[str], from_time : int, to_time : int):
    cur = conn.cursor()

    cur.execute(f"""
        SELECT n_gram,SUM(freq)
        FROM doc_freq NATURAL JOIN social_post_data
        WHERE n_gram IN ({','.join(['%s'] * len(ngrams))}) 
            AND create_utc >= %s AND create_utc < %s
        GROUP BY n_gram;
    """, ngrams + [from_time, to_time])

    items = cur.fetchall()
    items = {n_gram : freq for n_gram,freq in items}

    return [items[ngram] if ngram in items else 0 for ngram in ngrams]
    
def get_freq(ngrams : List[str], from_time : int, to_time : int):
    result = []
    with psycopg2.connect(POSTGRES_DB_URL) as conn:
        for i in range(0,len(ngrams),1000):
            print(i)
            batch = ngrams[i:i+1000]
            freq = get_batch_freq(conn, batch, from_time, to_time)
            result += freq
    
    return result

with psycopg2.connect(POSTGRES_DB_URL) as conn:
    cur = conn.cursor()

    cur.execute("""
        SELECT n_gram, SUM(freq)
        FROM doc_freq NATURAL JOIN social_post_data
        WHERE create_utc >= %s AND create_utc < %s
        GROUP BY n_gram, num_tokens;
    """, (current_time-recent_topic_seconds, current_time))

    candidate_topics = cur.fetchall()
    freq_after = [i[1] for i in candidate_topics]
    freq_before = get_freq([i[0] for i in candidate_topics], current_time - old_topic_seconds, current_time - recent_topic_seconds)

In [ ]:
from math import floor,e,factorial
filtered_topics = []

for (topic,_),fa,fb in zip(candidate_topics, freq_after, freq_before):
    pval = 1
    for x in range(0, floor(fa*num_recent/num_old + 1)):
        try:
            pmf = (fb if fb else 1)**x *  e**(-(fb if fb>0 else 1)) / factorial(x)
            pval -= pmf
        except OverflowError:
            continue
    if pval**len(topic.split(" ")) < 0.01:
        filtered_topics.append((topic, fa, fb, pval))
len(filtered_topics),filtered_topics

## Step 2: Manually insert into DB

In [ ]:
from typing import List
import random

def insert_emerging_topic(topic_name : str, create_time : int, date_start : int, date_end : int, regex: str, _retries=10, verbose=True):
    whitelist_characters = "qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM"
    with psycopg2.connect(POSTGRES_DB_URL) as conn:
        cur = conn.cursor()
        cur.execute("""
            SELECT MAX(topic_id) FROM emerging_topic;
        """)
        result = cur.fetchone()
        add_id = result[0]+1 if result[0]!=None else 0
        topic_key = "".join([random.choice(whitelist_characters) for _ in range(40)])

        try:
            cur.execute("""
                INSERT INTO emerging_topic (topic_id, topic_name, topic_key, create_utc, date_start, date_end)
                VALUES (%s, %s, %s, %s, %s, %s);
            """, (add_id, topic_name, topic_key, create_time, date_start, date_end))
        except psycopg2.IntegrityError as e:
            if verbose:
                print(f"Failed to insert {topic_name}. Retries left: {_retries}")
                print("   Message" + str(e))
            if _retries > 0: return insert_emerging_topic(topic_name, create_time, date_start, date_end, regex, _retries=_retries-1, verbose=verbose)
            else: raise Exception(f"Failed to insert {topic_name}. No more retries left")
        
        cur.execute("""
            INSERT INTO emerging_topic_regex (topic_id, regex)
            VALUES (%s, %s);
        """, (add_id, regex))
        conn.commit()
        cur.close()

In [ ]:
insert_emerging_topic("2024 US Election", current_time, current_time-old_topic_seconds, current_time-recent_topic_seconds, ".*(president)|( elect)|(harris)|(trump).*$")